# Mapping Public Disturbances in Chicago
### DataGood Final Project
### Esme Cohen and Cameron Fozi
#### Leads: Isabella Borkovic and Gloria Zhang

In [ ]:
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.listdir("../input"))

In [ ]:
df = pd.read_csv('../input/Chicago_Crimes_2012_to_2017.csv')
df.head()

# Getting rid of null values!

In [ ]:
df = df.dropna()
df = df.drop(columns=['Unnamed: 0', 'ID', 'Case Number', 'Block', 'IUCR','Domestic', 'Beat', 'District', 'Ward','X Coordinate', 'Y Coordinate','Updated On', 'FBI Code'], axis = 1)

# Knitty Gritty?

In [ ]:
pd.value_counts(df['Location Description'])[:10]

In [ ]:
pd.value_counts(df['Primary Type'])[:10]

In [ ]:
chicago_map = folium.Map(location=[41.864073,-87.706819],
                        zoom_start=11,
                        tiles="CartoDB dark_matter")

In [ ]:
locations = df.groupby('Community Area').first()
new_locations = locations.loc[:, ['Latitude', 'Longitude', 'Location Description', 'Arrest']]
new_locations.head()

In [ ]:
popup_text = """Community Index : {}<br
                Arrest : {}<br>
                Location Description : {}<br>"""

# Time to Map

In [ ]:
for i in range(len(new_locations)):
    lat = new_locations.iloc[i][0]
    long = new_locations.iloc[i][1]
    popup_text = """Community Index : {}<br>
                Arrest : {}<br>
                Location Description : {}<br>"""
    popup_text = popup_text.format(new_locations.index[i],
                               new_locations.iloc[i][-1],
                               new_locations.iloc[i][-2]
                               )
    folium.CircleMarker(location = [lat, long], popup= popup_text, fill = True).add_to(chicago_map)

In [ ]:
unique_locations = df['Location'].value_counts()
unique_locations.index

In [ ]:
def Location_extractor(Raw_Str):
    preProcess = Raw_Str[1:-1].split(',')
    lat =  float(preProcess[0])
    long = float(preProcess[1])
    return (lat, long)

In [ ]:
CR_index = pd.DataFrame({"Raw_String" : unique_locations.index, "ValueCount":unique_locations})
CR_index.index = range(len(unique_locations))
CR_index['LocationCoord'] = CR_index['Raw_String'].apply(Location_extractor)
CR_index  = CR_index.drop(columns=['Raw_String'], axis = 1)


In [ ]:
chicago_map_crime = folium.Map(location=[41.895140898, -87.624255632],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")

for i in range(500):
    lat = CR_index['LocationCoord'].iloc[i][0]
    long = CR_index['LocationCoord'].iloc[i][1]
    radius = CR_index['ValueCount'].iloc[i] / 45
    
    if CR_index['ValueCount'].iloc[i] > 1000:
        color = "#FF2D00"
    elif 800 < CR_index['ValueCount'].iloc[i] <= 1000:
        color = '#FF8F00'
    elif 600 < CR_index['ValueCount'].iloc[i] <= 800:
        color = '#FFF700'
    elif 400 < CR_index['ValueCount'].iloc[i] <= 600:
        color = '#64FF00'
    elif 200 < CR_index['ValueCount'].iloc[i] <= 400:
        color = '#003EFF' 
    else:
        color = "#D100FF"
    
    popup_text = """Latitude : {}<br>
                Longitude : {}<br>
                Criminal Incidents : {}<br>"""
    popup_text = popup_text.format(lat,
                               long,
                               CR_index['ValueCount'].iloc[i]
                               )
    folium.CircleMarker(location = [lat, long], popup= popup_text,radius = radius, color = color, fill = True).add_to(chicago_map_crime)

In [ ]:
chicago_map_crime

In [ ]:
pdc = df['Primary Type'] == 'PUBLIC PEACE VIOLATION'
pdc_df = df[pdc]['Location'].value_counts()
pdc_index = pd.DataFrame({"Raw_String" : pdc_df.index, "ValueCount":pdc_df}).head(500)
pdc_index['LocationCoord'] = pdc_index['Raw_String'].apply(Location_extractor)

In [ ]:
pdc_index['LocationCoord'] = pdc_index['Raw_String'].apply(Location_extractor)

In [ ]:
chicago_map_pdc = folium.Map(location=[41.895140898, -87.624255632],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")

for i in range(500):
    lat = pdc_index['LocationCoord'].iloc[i][0]
    long = pdc_index['LocationCoord'].iloc[i][1]
    radius = pdc_index['ValueCount'].iloc[i] / 1.5
    
    if pdc_index['ValueCount'].iloc[i] >= 25:
        color = "#FF00E6"
    elif 10 <= pdc_index['ValueCount'].iloc[i] < 24:
        color = "#FF79F2"
    else:
        color = "#FFE4FC"
    
    popup_text = """Latitude : {}<br>
                Longitude : {}<br>
                Public Peace Disturbance : {}<br>"""
    popup_text = popup_text.format(lat,
                               long,
                               pdc_index['ValueCount'].iloc[i])
    folium.CircleMarker(location = [lat, long], popup= popup_text,radius = radius, color = color, fill = True).add_to(chicago_map_pdc)

In [ ]:
chicago_map_pdc

# But, what's a public peace violation?

In [ ]:
df_public_peace =  df[df['Primary Type'] == 'PUBLIC PEACE VIOLATION']

In [ ]:
df_public_data = pd.DataFrame({"Counts": df_public_peace['Description'].value_counts(), "Description" : df_public_peace['Description'].value_counts().index})
df_public_data.reset_index(inplace=True)
df_public_data = df_public_data.drop(columns=['index'], axis = 1)
df_public_data.head()

In [ ]:
plt.figure(figsize = (15, 7))
sns.barplot(y ="Description", x = "Counts", data = df_public_data, palette= "rainbow")

# Arrest and Reckless Conduct

In [ ]:
only_rc = df[df['Description'] == 'RECKLESS CONDUCT']
rc_by_ca = only_rc['Community Area'].value_counts().to_frame().reset_index().rename(columns={'index':'Community Area', 'Community Area': 'RC Number'}).sort_values(by = "Community Area", ascending = True)
rc_by_ca.head()

In [ ]:
arrest_only_rc = only_rc[only_rc['Arrest'] == True]['Community Area'].value_counts().to_frame().reset_index().rename(columns={'index':'Community Area', 'Community Area': 'RC Arrest Number'}).sort_values(by = "Community Area", ascending = True)
arrest_only_rc.head()

In [ ]:
new = arrest_only_rc['RC Arrest Number'] / rc_by_ca['RC Number']
neww = new.to_frame().reset_index().rename(columns={'index':'Community Area', 0: '% Resulting in Arrest'})
neww['Community Area'] = neww['Community Area'] + 1
neww.sort_values(by = "Community Area", ascending = True).set_index('Community Area').head(10)

In [ ]:
lst = [24.48, 13.37, 17.52, 3.58, 1.66, 3.60, 4.56, 8.13, 1.07, 0.92, 1.10, 0.96, 3.07, 4.97, 1.41, 3.04, 2.41, 4.43, 2.34, 1.49, 1.96, 4.62, 40.87, 7.01, 83.0, 95.60, 91.28, 28.08, 90.39, 11.05, 3.5, 8.93, 24.56, 10.11, 68.22, 92.59, 89.12, 92.56, 67.34, 91.62, 26.84, 82.93, 93.47, 96.61, 97.4, 74.10, 97.50, 93.47, 96.4, 82.76, 67.15, 2.88, 93.39, 95.3, 6.30, 4.4, 0.94, 83.12, 0.98, 2.58, 23.61, 1.50, 3.82, 0.56, 3.26, 46.98, 91.2, 94.12, 96.07, 48.29, 96.86, 34.9, 95.8, 3.54, 65.98, 1.22, 13.48]

In [ ]:
neww['% Black'] = lst
neww.head()

In [ ]:
neww.corr()